In [14]:
# !pip install dotenv fyers_api
from fyers_api import fyersModel
from fyers_api import accessToken
import pandas as pd
import datetime as dt
import os
from dotenv import load_dotenv
load_dotenv("../env.env")
# print("running")

True

In [15]:
client_id=os.environ.get('API_CLIENT_ID')
secret_key=os.environ.get('API_SECRET_KEY')
redirect_uri=os.environ.get('API_REDIRECT_URL')


In [16]:
session=accessToken.SessionModel(client_id=client_id,
secret_key=secret_key,redirect_uri=redirect_uri, 
response_type='code', grant_type='authorization_code')

In [17]:
auth_code_provider = session.generate_authcode()
print(auth_code_provider)

https://api.fyers.in/api/v2/generate-authcode?client_id=QW4JS7GAG2-100&redirect_uri=https%3A%2F%2F127.0.0.1%3A5000%2Flogin&response_type=code&state=None


In [18]:
auth_code=str(input("Enter the auth_code: "))

In [19]:
session.set_token(auth_code)
response = session.generate_token()
access_token = response["access_token"]
if(access_token is None):
    print("Access token is not generated")
else:
    print("Access token is generated")
    

KeyError: 'access_token'

In [ ]:
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token)
is_async = True

In [ ]:
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'XA15265',
  'name': 'AIYAZ ANWAR QURESHI',
  'image': None,
  'display_name': None,
  'pin_change_date': '31-07-2022 00:55:39',
  'email_id': 'qureshi.aiyaz123@gmail.com',
  'pwd_change_date': '31-07-2022 00:54:23',
  'PAN': '---------',
  'pwd_to_expire': 149}}

In [ ]:
# Getting Data
def dynamicDataFetching(startDate,endDate,symbol):
    # Creating a dataframe to store the data
    df = pd.DataFrame(columns=["date","open","high","low","close","volume"])
    today = dt.date.today()
    range_from=startDate
    while(range_from<=today):
    # Check whether the upcoming endDate is lesser than today if we add 99D or else make it today
        if((range_from + dt.timedelta(days=99))<today):
            range_to = range_from + dt.timedelta(days=99)
        else:
            range_to = today

        # Api call 
        data = {"symbol":symbol,"resolution":"5","date_format":"1","range_from":range_from,"range_to":range_to,"cont_flag":"1"}
        res = fyers.history(data)
        res = res["candles"]
        
        # Converting the data into pandas dataframe 
        tempDf = pd.DataFrame(columns=["date","open","high","low","close","volume"])
        for i in range(len(res)):
             tempDf.loc[i]= [
                res[i][0],
                res[i][1],
                res[i][2],
                res[i][3],
                res[i][4],
                res[i][5]]
       
        # Appending the data to the main dataframe
        df = pd.concat([df,tempDf])

        # Updating the range_from to last date of last api call
        range_from=range_to+dt.timedelta(days=1)
    return df

In [ ]:
startDate = dt.date(2021,7,1)
endDate = dt.date.today()
symbol="NSE:TATAMOTORS-EQ"

df = dynamicDataFetching(startDate,endDate,symbol)
print(df.dtypes)
print(df.describe())

date      object
open      object
high      object
low       object
close     object
volume    object
dtype: object
                date     open     high      low    close    volume
count   2.026000e+04  20260.0  20260.0  20260.0  20260.0   20260.0
unique  2.026000e+04   4147.0   4046.0   4067.0   4162.0   19893.0
top     1.625111e+09    432.0    432.5    432.5    432.0  246975.0
freq    1.000000e+00     25.0     35.0     33.0     26.0       3.0


In [20]:
print(df.head())

           date    open    high     low  close     volume
0  1625111100.0   341.5   343.2   341.2  342.1  1814940.0
1  1625111400.0  341.95  344.95   341.6  344.4  1647938.0
2  1625111700.0   344.3   345.2   343.6  343.9  1009616.0
3  1625112000.0   344.0   344.7   343.6  343.8   587118.0
4  1625112300.0   343.7   344.2  343.65  344.0   290437.0


In [21]:
# Converting everything to playable format + changing the time zone from utc to ist
df["date"]= pd.to_datetime(df["date"],unit='s',utc=True).dt.tz_convert('Asia/Kolkata')
df.set_index("date",inplace=True)
df["open"]=df["open"].astype(float)
df["high"]=df["high"].astype(float)
df["low"]=df["low"].astype(float)
df["close"]=df["close"].astype(float)
df["volume"]=df["volume"].astype(float)

print(df.head())
print("\nData types :\n" , df.dtypes)

                             open    high     low  close     volume
date                                                               
2021-07-01 09:15:00+05:30  341.50  343.20  341.20  342.1  1814940.0
2021-07-01 09:20:00+05:30  341.95  344.95  341.60  344.4  1647938.0
2021-07-01 09:25:00+05:30  344.30  345.20  343.60  343.9  1009616.0
2021-07-01 09:30:00+05:30  344.00  344.70  343.60  343.8   587118.0
2021-07-01 09:35:00+05:30  343.70  344.20  343.65  344.0   290437.0

Data types :
 open      float64
high      float64
low       float64
close     float64
volume    float64
dtype: object


In [22]:
# Export df to csv
fileName = "TataMotors.csv"
# df.to_csv(f"../data/{fileName}")